In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points
from geopy.distance import distance, lonlat
from tqdm.notebook import tqdm
import xml.etree.ElementTree as ET
from ast import literal_eval
import re
import pickle


In [ ]:
# Creating the DataFrame of the earthquakes' probabilities
df = pd.read_table('../incremental_annual_rates/incremental_annual_rates_italy.txt', sep='\s+', header=None)

df.columns = ['Lat', 'Long'] + [str(np.around(x,1)) for x in np.arange(4.6, 8.7, 0.1)]
df.drop(df.columns[42:], axis=1, inplace=True)
df.fillna(value=0.0, inplace=True)
df = df[df['4.6'] != 0.0]
df.rename(columns={'Lat':'Long', 'Long':'Lat'}, inplace=True)

df.to_csv('incremental_annual_rates/incremental_annual_rates_italy_without_zeros.csv', index=False)

In [18]:
df = pd.read_csv('../incremental_annual_rates/incremental_annual_rates_italy.csv')
#df.set_index(['Lat','Long'], inplace=True)
df.head(2)

,Lat,Long,4.6,4.7,4.8,4.9,5.0,5.1,5.2,5.3,...,7.7,7.8,7.9,8.0,8.1,8.2,8.3,8.4,8.5,8.6
0,5.00,48.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.05,48.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df['Lat'] = (df['Lat']*5).astype(int)
df['Long'] = (df['Long']*5).astype(int)
df_4 = df.groupby(['Lat','Long']).sum()
df_4 = df_4[df_4['4.6'] != 0.0]
df_4.reset_index(inplace=True)
df_4['Lat'] = df_4['Lat']/5
df_4['Long'] = df_4['Long']/5
df_4

,Lat,Long,4.6,4.7,4.8,4.9,5.0,5.1,5.2,5.3,...,7.7,7.8,7.9,8.0,8.1,8.2,8.3,8.4,8.5,8.6
0,5.0,45.0,5.268220e-09,4.184696e-09,3.324022e-09,2.640365e-09,2.097316e-09,1.665957e-09,1.323317e-09,1.051148e-09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.0,45.2,2.802193e-08,2.225861e-08,1.768064e-08,1.404423e-08,1.115573e-08,8.861313e-09,7.038791e-09,5.591110e-09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,45.4,5.112818e-08,4.061256e-08,3.225970e-08,2.562479e-08,2.035450e-08,1.616815e-08,1.284282e-08,1.020141e-08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.0,45.6,1.131072e-07,8.984425e-08,7.136582e-08,5.668789e-08,4.502879e-08,3.576764e-08,2.841125e-08,2.256786e-08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,45.8,2.701120e-07,2.145576e-07,1.704292e-07,1.353767e-07,1.075335e-07,8.541693e-08,6.784908e-08,5.389444e-08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3174,19.8,39.4,1.269970e-07,1.008773e-07,8.012970e-08,6.364928e-08,5.055842e-08,4.015998e-08,3.190021e-08,2.533923e-08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3175,19.8,39.6,1.615555e-07,1.283281e-07,1.019346e-07,8.096956e-08,6.431640e-08,5.108834e-08,4.058091e-08,3.223456e-08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3176,19.8,39.8,1.051104e-07,8.349213e-08,6.632016e-08,5.267997e-08,4.184519e-08,3.323882e-08,2.640253e-08,2.097228e-08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3177,19.8,40.0,2.083151e-08,1.654706e-08,1.314380e-08,1.044049e-08,8.293175e-09,6.587503e-09,5.232639e-09,4.156433e-09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
df_4.to_csv('earthquake_probabilities/italy_ds16.csv', index=False)

In [21]:
g = nx.read_gml('networks/italy.gml', label='id')

In [22]:
def intensity_europe(M, R):
    h = 3.91
    D = np.sqrt(h**2 + R**2)
    return 1.621*M - 1.343 - 0.0086*(D-h) - 1.037*(np.log(D)-np.log(h))

In [ ]:
l = [tuple(d.values()) for d in g.edges[('Rome', 'Pescara', 0)]['points']['point']]
edge = LineString(l)
point = Point(10.8,48.3)
nearest_point = nearest_points(edge, point)[0]
nearest_point.coords[:]

In [ ]:
for e in g.edges:
    edge_points = [tuple(p.values()) for p in g.edges[e]['points']['point']]
    edge = LineString(edge_points)
    epicenter = Point(10.5,48.2)
    nearest_point = nearest_points(edge, epicenter)[0].coords[0]
    print(nearest_point)
    print(epicenter.coords[0])
    dist = distance(lonlat(*epicenter.coords[0]), lonlat(*nearest_point)).km
    intensity = intensity_europe(8.6, dist)
    print("%.1f" % intensity)

In [24]:
#df['Epicenter'] = list(zip(df.Lat, df.Long))
df.insert(loc=0, column='Epicenter', value=list(zip(df.Lat, df.Long)))
df.set_index('Epicenter', inplace=True)
df.drop(['Lat', 'Long'], axis=1, inplace=True)

In [25]:
#df=df_4
df.head()

,4.6,4.7,4.8,4.9,5.0,5.1,5.2,5.3,5.4,5.5,...,7.7,7.8,7.9,8.0,8.1,8.2,8.3,8.4,8.5,8.6
Epicenter,,,,,,,,,,,,,,,,,,,,,
"(5.0, 45.0)",5.268220e-09,4.184696e-09,3.324022e-09,2.640365e-09,2.097316e-09,1.665957e-09,1.323317e-09,1.051148e-09,8.349566e-10,6.632296e-10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(5.0, 45.2)",2.802193e-08,2.225861e-08,1.768064e-08,1.404423e-08,1.115573e-08,8.861313e-09,7.038791e-09,5.591110e-09,4.441177e-09,3.527752e-09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(5.0, 45.4)",5.112818e-08,4.061256e-08,3.225970e-08,2.562479e-08,2.035450e-08,1.616815e-08,1.284282e-08,1.020141e-08,8.103271e-09,6.436657e-09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(5.0, 45.6)",1.131072e-07,8.984425e-08,7.136582e-08,5.668789e-08,4.502879e-08,3.576764e-08,2.841125e-08,2.256786e-08,1.792628e-08,1.423935e-08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(5.0, 45.8)",2.701120e-07,2.145576e-07,1.704292e-07,1.353767e-07,1.075335e-07,8.541693e-08,6.784908e-08,5.389444e-08,4.280987e-08,3.400509e-08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
epicenters = df.index.to_numpy()
magnitudes = df.columns.to_numpy()

In [27]:
I = np.ones((g.number_of_edges(), len(epicenters), len(magnitudes)))
for i, e in tqdm(enumerate(g.edges), total=g.number_of_edges()):
    edge_points = [tuple(p.values()) for p in g.edges[e]['points']['point']]
    l = LineString(edge_points)
    for j, epicenter in tqdm(enumerate(epicenters), total=len(epicenters)):
        p = Point(epicenter)
        nearest_point = nearest_points(l, p)[0].coords[0]
        R = distance(lonlat(*epicenter), lonlat(*nearest_point)).km
        for k, M in enumerate(magnitudes):
            intensity = intensity_europe(np.float(M), R)
            if intensity > 1.0:
                I[i,j,k] = intensity

In [28]:
np.save('intensities/italy_ds8.npy', I)

In [ ]:
def remains_connected(g, srlg):
    g.remove_edges_from(srlg)
    return nx.is_connected(g.to_undirected())

def get_minimal_cut_SRLGs(PSRLG_file)
    xtree = ET.parse(PSRLG_file)
    xroot = xtree.getroot()
    cut_srlgs = []
    for fs in tqdm(xroot.iter('Failure_State'), total=len(xroot.findall('Failure_State'))):
        # (edge_id, node_id, node_id)
        srlg = set([ (int(edge[1]), int(edge[2]), 0) if edge[0] != '24' else (int(edge[1]), int(edge[2]), 1) for edge in re.findall(r"(\d+):\((\d+)\D+(\d+)\D+\)", fs.find('Edges').text)])
        if not remains_connected(g.copy(), srlg):
            added = False
            for i, s in tqdm(enumerate(cut_srlgs), total=len(cut_srlgs)):
                if srlg.issubset(s):
                    cut_srlgs[i] = srlg
                    added = True
                    break
                elif srlg.issuperset(s):
                    added = True
                    break
            if not cut_srlgs or not added:
                cut_srlgs.append(srlg)
        return cut_srlgs

cut_srlgs = get_minimal_cut_SRLGs('PSRLGs/whole_graph_simple_VI_upper_big_grid.xml')

In [ ]:
# Write
with open('min_cut_SRLGs/italy_corrected', 'wb') as fp:
    pickle.dump(cut_srlgs, fp)

# Read
#with open ('min_cut_SRLGs/italy', 'rb') as fp:
#    cut_srlgs = pickle.load(fp)

In [ ]:
edge_list = list(g.edges)
for idx,srlg in enumerate(cut_srlgs):
    for l in srlg:
        if l not in edge_list:
            cut_srlgs[idx].discard(l)
            cut_srlgs[idx].add((l[1],l[0],l[2]))

In [ ]:
cut_srlgs